In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OrdinalEncoder

In [2]:
data = pd.read_csv('diabetic_data.csv')

data.describe()
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [3]:
def print_unique_elements(df, columns):
    for column in columns:
        try:
            print(f"\033[94m{column}\033[0m:")
            unique_counts = df[column].value_counts()
            i = 0
            for value, count in unique_counts.items():
                i = i + 1
                print(f"        \033[91m{value}\033[0m {count}")
        except KeyError:
            print(f"    Column '{column}' not found in the DataFrame.")

In [4]:
print (data.columns)

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')


In [13]:
data['admission_type_id'].replace([5,6,8], np.nan, inplace=True)
data['discharge_disposition_id'].replace([18,25,26], np.nan, inplace=True)
data['admission_source_id'].replace([9,15,17,20, 21], np.nan, inplace=True)

In [14]:

def print_missing_counts(df, null_values):
    for col in df.select_dtypes(include=['object']).columns:
        
        for val in null_values :
            count = df[col].value_counts().get(val, 0)
            if count > 0:
                print(f"The count of {val} in the column '{col}' is: {count} equals to {((count/df.shape[0])*100):.2f}% missing values.")

    nas = df.isna().sum()
    nas = nas[nas > 0]
    nas_percentage = nas / len(df) * 100
    for column in nas.index:
        print(f"{column}: Count = {nas[column]}, Percentage = {nas_percentage[column]:.2f}%")

print_missing_counts(data, ['?', 'Unknown/Invalid'])

The count of ? in the column 'race' is: 2273 equals to 2.23% missing values.
The count of Unknown/Invalid in the column 'gender' is: 3 equals to 0.00% missing values.
The count of ? in the column 'weight' is: 98569 equals to 96.86% missing values.
The count of ? in the column 'payer_code' is: 40256 equals to 39.56% missing values.
The count of ? in the column 'medical_specialty' is: 49949 equals to 49.08% missing values.
The count of ? in the column 'diag_1' is: 21 equals to 0.02% missing values.
The count of ? in the column 'diag_2' is: 358 equals to 0.35% missing values.
The count of ? in the column 'diag_3' is: 1423 equals to 1.40% missing values.
admission_type_id: Count = 10396, Percentage = 10.22%
discharge_disposition_id: Count = 4680, Percentage = 4.60%
admission_source_id: Count = 7067, Percentage = 6.94%
max_glu_serum: Count = 96420, Percentage = 94.75%
A1Cresult: Count = 84748, Percentage = 83.28%


In [15]:
print_unique_elements(data, ['tolbutamide'])

tolbutamide:
        No 101743
        Steady 23


In [85]:
no_missing = data[data['gender'] != 'Unknown/Invalid']
no_missing['gender'].unique()

array(['Female', 'Male'], dtype=object)

this fields have significant number of missing values and dosen't have high importance in data so we can ignore them .

In [86]:
no_missing = no_missing.drop(columns=['max_glu_serum', 'A1Cresult', 'payer_code', 'weight',  'medical_specialty'])

but medical_specialty has high importance in data so we imputed it despite significant number of missing values .

In [87]:
def drop_columns(df, distinct_percentage, value_proportion_percentage):
    for col in df.columns:
        if df[col].nunique() / len(df) > distinct_percentage:
            print(f'column {col} dropped cause it has more than {distinct_percentage * 100}% unique values!')
            df = df.drop(col, axis=1)

    for col in df.columns:
        if any((df[col].value_counts() / len(df)) > value_proportion_percentage):
            print(f'column {col} dropped cause it has a value by more than {value_proportion_percentage * 100}% proportion!')
            df = df.drop(col, axis=1)

    return df

no_missing = drop_columns(no_missing, 0.90, 0.99)
no_missing.info()

column encounter_id dropped cause it has more than 90.0% unique values!
column nateglinide dropped cause it has a value by more than 99.0% proportion!
column chlorpropamide dropped cause it has a value by more than 99.0% proportion!
column acetohexamide dropped cause it has a value by more than 99.0% proportion!
column tolbutamide dropped cause it has a value by more than 99.0% proportion!
column acarbose dropped cause it has a value by more than 99.0% proportion!
column miglitol dropped cause it has a value by more than 99.0% proportion!
column troglitazone dropped cause it has a value by more than 99.0% proportion!
column tolazamide dropped cause it has a value by more than 99.0% proportion!
column examide dropped cause it has a value by more than 99.0% proportion!
column citoglipton dropped cause it has a value by more than 99.0% proportion!
column glyburide-metformin dropped cause it has a value by more than 99.0% proportion!
column glipizide-metformin dropped cause it has a value 

In [88]:
no_missing = no_missing.replace('?', np.nan)

In [89]:
print_missing_counts(no_missing, ['?', 'Unknown/Invalid'])

race: Count = 2271, Percentage = 2.23%
admission_type_id: Count = 10396, Percentage = 10.22%
discharge_disposition_id: Count = 4680, Percentage = 4.60%
admission_source_id: Count = 7067, Percentage = 6.94%
diag_1: Count = 21, Percentage = 0.02%
diag_2: Count = 358, Percentage = 0.35%
diag_3: Count = 1423, Percentage = 1.40%


In [90]:
print_missing_counts(no_missing, ['?'])

race: Count = 2271, Percentage = 2.23%
admission_type_id: Count = 10396, Percentage = 10.22%
discharge_disposition_id: Count = 4680, Percentage = 4.60%
admission_source_id: Count = 7067, Percentage = 6.94%
diag_1: Count = 21, Percentage = 0.02%
diag_2: Count = 358, Percentage = 0.35%
diag_3: Count = 1423, Percentage = 1.40%


In [91]:
import gc

def label_encode(df):
    label_encoders = {}

    object_columns = df.select_dtypes(include=['object']).columns

    for column in object_columns:
        le = LabelEncoder()
        
        non_null_mask = df[column].notnull()
        non_null_values = df[column][non_null_mask]
        
        encoded_values = le.fit_transform(non_null_values)
        
        full_encoded = pd.Series(index=df[column].index, data=np.nan, dtype=float)
        
        full_encoded[non_null_mask] = encoded_values
        
        df[column] = full_encoded
        
        del full_encoded
        
        gc.collect()
        
        print(f"{column} labels: ", dict(zip(le.classes_, le.transform(le.classes_))))
        label_encoders[column] = le

    return df, label_encoders


In [92]:
no_missing , label_encoders = label_encode(no_missing)

race labels:  {'AfricanAmerican': 0, 'Asian': 1, 'Caucasian': 2, 'Hispanic': 3, 'Other': 4}
gender labels:  {'Female': 0, 'Male': 1}
age labels:  {'[0-10)': 0, '[10-20)': 1, '[20-30)': 2, '[30-40)': 3, '[40-50)': 4, '[50-60)': 5, '[60-70)': 6, '[70-80)': 7, '[80-90)': 8, '[90-100)': 9}
diag_1 labels:  {'10': 0, '11': 1, '110': 2, '112': 3, '114': 4, '115': 5, '117': 6, '131': 7, '133': 8, '135': 9, '136': 10, '141': 11, '142': 12, '143': 13, '145': 14, '146': 15, '147': 16, '148': 17, '149': 18, '150': 19, '151': 20, '152': 21, '153': 22, '154': 23, '155': 24, '156': 25, '157': 26, '158': 27, '160': 28, '161': 29, '162': 30, '163': 31, '164': 32, '170': 33, '171': 34, '172': 35, '173': 36, '174': 37, '175': 38, '179': 39, '180': 40, '182': 41, '183': 42, '184': 43, '185': 44, '187': 45, '188': 46, '189': 47, '191': 48, '192': 49, '193': 50, '194': 51, '195': 52, '196': 53, '197': 54, '198': 55, '199': 56, '200': 57, '201': 58, '202': 59, '203': 60, '204': 61, '205': 62, '207': 63, '208

In [93]:
cop = no_missing.copy()

In [94]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

def impute_missing_values(df, columns_with_missing):
    for column in columns_with_missing:
        print (f'Imputing {column}')
        gc.collect()
        target_column = column
        not_null_df = df[df[target_column].notnull()]
        null_df = df[df[target_column].isnull()]

        X_train = not_null_df.drop(target_column, axis=1)
        y_train = not_null_df[target_column]
        X_test = null_df.drop(target_column, axis=1)
        
        if not X_test.empty:
            model = RandomForestClassifier()
            model.fit(X_train, y_train)

            predictions = model.predict(X_test)
            df.loc[df[target_column].isnull(), target_column] = predictions
    return df

no_missing = impute_missing_values(no_missing, ['race', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id'])
print_missing_counts(no_missing, ['?'])


Imputing race
Imputing admission_type_id
Imputing discharge_disposition_id
Imputing admission_source_id
diag_1: Count = 21, Percentage = 0.02%
diag_2: Count = 358, Percentage = 0.35%
diag_3: Count = 1423, Percentage = 1.40%


In [96]:
from sklearn.impute import KNNImputer

def impute_categorical_missing_values_knn(df, n_neighbors=5):
    imputer = KNNImputer(n_neighbors=n_neighbors)

    df_imputed = imputer.fit_transform(df)

    df_imputed = pd.DataFrame(df_imputed, columns=df.columns)

    return df_imputed

no_missing = impute_categorical_missing_values_knn(no_missing , 1)
print_missing_counts(no_missing, ['?'])


In [98]:
no_missing.drop_duplicates(subset=None, keep='first', inplace=True)

In [99]:
no_missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101763 entries, 0 to 101762
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   patient_nbr               101763 non-null  float64
 1   race                      101763 non-null  float64
 2   gender                    101763 non-null  float64
 3   age                       101763 non-null  float64
 4   admission_type_id         101763 non-null  float64
 5   discharge_disposition_id  101763 non-null  float64
 6   admission_source_id       101763 non-null  float64
 7   time_in_hospital          101763 non-null  float64
 8   num_lab_procedures        101763 non-null  float64
 9   num_procedures            101763 non-null  float64
 10  num_medications           101763 non-null  float64
 11  number_outpatient         101763 non-null  float64
 12  number_emergency          101763 non-null  float64
 13  number_inpatient          101763 non-null  f

this is an alternative way to impute the missing values for categorical features

no_missing.loc[:, 'race'] = no_missing['race'].replace('?', np.nan)

df_encoded = pd.get_dummies(no_missing)

imputer = KNNImputer(n_neighbors=3)
df_imputed = imputer.fit_transform(df_encoded)

df_imputed = pd.DataFrame(df_imputed, columns = df_encoded.columns)

no_missing.loc[:, 'race'] = df_imputed[['race_Caucasian', 'race_AfricanAmerican', 'race_Hispanic', 'race_Other', 'race_Asian']].idxmax(axis=1).str.replace('race_', '')
mode_value = no_missing['race'].mode()[0]
no_missing.loc[:, 'race'] = no_missing['race'].fillna(mode_value)
print_unique_elements(no_missing , ['race'])

now we try to detect outliers

In [124]:
from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np

def detect_outliers(df, n_estimators, contamination, random_state):
    print (f'in outlier detection n_estimators is {n_estimators} and the contamination is {contamination}')
    df_copy = df.copy()

    iso = IsolationForest(n_estimators=n_estimators, contamination=contamination ,random_state=random_state)

    outlier_label = iso.fit_predict(df_copy)

    df_copy['outlier'] = outlier_label

    df_outliers = df_copy[df_copy['outlier'] == -1]

    df_copy = df_copy[df_copy['outlier'] == 1]
    df_copy.drop('outlier', axis=1, inplace=True)

    return df_copy, df_outliers

df_no_outliers, df_outliers = detect_outliers(no_missing, 50, 0.05,42)


in outlier detection n_estimators is 50 and the contamination is 0.05


lets normalize the data.

In [101]:
from sklearn.preprocessing import StandardScaler

def normalize_features(df):
    df_copy = df.copy()

    scaler = StandardScaler()

    df_normalized = pd.DataFrame(scaler.fit_transform(df_copy), columns=df_copy.columns)

    return df_normalized

df_normalized = normalize_features(df_no_outliers)


now lets go for dimentionality reduction step !

In [140]:
from sklearn.decomposition import PCA

def apply_pca(df, variance_ratio, random_state):
    print(f'in pca the variance ratio is {variance_ratio}')
    df_copy = df.copy()

    pca = PCA(n_components=variance_ratio, random_state=random_state)

    df_pca = pd.DataFrame(pca.fit_transform(df_copy))
    
    if (variance_ratio < 1):
        print (f'pca dimontions after pca : {df_pca.shape[1]}')
    else: 
        print(f'Total explained variance: {sum(pca.explained_variance_ratio_)}')
    
    return df_pca

variance_ratio = 0.95  
df_pca = apply_pca(df_normalized, 5, 42)


in pca the variance ratio is 5
Total explained variance: 0.314868627517442


In [118]:
num_dimensions = df_pca.shape[1]
print(f"The dimensionality of the data after PCA is: {num_dimensions}")

The dimensionality of the data after PCA is: 5


now let's implement the K-means clustering function.

In [158]:
from sklearn.cluster import KMeans

def fit_kmeans(df, n_clusters, random_state):
    print (f'in k-means the n_clusters is {n_clusters}')
    df_copy = df.copy()

    kmeans = KMeans(n_clusters=n_clusters, n_init='auto', init='k-means++', random_state=random_state)

    kmeans.fit(df_copy)

    df_copy['cluster'] = kmeans.labels_

    return df_copy, kmeans

n_clusters = 10
K_means_df , K_means_modle = fit_kmeans(df_pca , n_clusters, 42)

in k-means the n_clusters is 10


and also let's implement the DBScan clustering function

In [157]:
from sklearn.cluster import DBSCAN

def fit_dbscan(df, eps, min_samples):
    df_copy = df.copy()

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)

    dbscan.fit(df_copy)

    df_copy['cluster'] = dbscan.labels_

    return df_copy, dbscan

eps = 0.1
min_samples = 10
dbscan_df , dbscan_modle = fit_dbscan(df_pca , eps, min_samples)


now lets fine best hyper parameters

In [119]:
from tqdm import tqdm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import IsolationForest

class kmeans_pipe_hyper_parameter_tuner:
    
    def __init__(self, n_estimators, contamination, variance_ratio, n_clusters, df, random_state):
        self.n_estimators = n_estimators
        self.contamination = contamination
        self.variance_ratio = variance_ratio
        self.n_clusters = n_clusters
        self.df = df
        self.random_state=random_state
        
    def fit(self):
        df_no_outliers, df_outliers = detect_outliers(self.df, self.n_estimators, self.contamination, self.random_state)
        df_normalized = normalize_features(df_no_outliers)
        df_pca = apply_pca(df_normalized, self.variance_ratio, self.random_state)
        K_means_df , K_means_modle = fit_kmeans(df_pca , n_clusters, self.random_state)
        self.model = K_means_modle
        self.PCA_df = df_pca
        
    


In [122]:
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score
from tqdm import tqdm

param_grid = {
    'n_estimators': [10, 25, 50],
    'contamination': [0.01, 0.05, 0.1],
    'variance_ratio': [0.9, 0.95],
    'n_clusters': [3, 5, 6, 8, 9, 10]
}

grid = ParameterGrid(param_grid)

best_score = -1
best_params = None

pbar = tqdm(total=len(grid), desc='Hyperparameter Tuning', ncols=80)

for params in grid:
    tuner = kmeans_pipe_hyper_parameter_tuner(**params, df=no_missing, random_state=42)
    tuner.fit()
    score = silhouette_score(tuner.PCA_df, tuner.model.labels_)
    
    print (f'for params {params} got the score of : {score}')
    
    if score > best_score:
        best_score = score
        best_params = params

    pbar.update()

pbar.close()

print(f"Best score: {best_score}")
print(f"Best parameters: {best_params}")

Hyperparameter Tuning:   1%|▏                | 1/108 [02:19<4:08:58, 139.62s/it]

for params {'contamination': 0.01, 'n_clusters': 3, 'n_estimators': 10, 'variance_ratio': 0.9} got the score of : 0.03759970116863179


Hyperparameter Tuning:   2%|▎                | 2/108 [04:39<4:06:48, 139.70s/it]

for params {'contamination': 0.01, 'n_clusters': 3, 'n_estimators': 10, 'variance_ratio': 0.95} got the score of : 0.03759970116863179


Hyperparameter Tuning:   3%|▍                | 3/108 [07:02<4:07:06, 141.21s/it]

...

for params {'contamination': 0.1, 'n_clusters': 10, 'n_estimators': 25, 'variance_ratio': 0.95} got the score of : 0.04277768598735495


Hyperparameter Tuning:  99%|██████████████▊| 107/108 [3:50:03<01:51, 111.91s/it]

for params {'contamination': 0.1, 'n_clusters': 10, 'n_estimators': 50, 'variance_ratio': 0.9} got the score of : 0.057534251634545786


Hyperparameter Tuning: 100%|███████████████| 108/108 [3:51:56<00:00, 128.86s/it]

for params {'contamination': 0.1, 'n_clusters': 10, 'n_estimators': 50, 'variance_ratio': 0.95} got the score of : 0.057534251634545786
Best score: 0.057534251634545786
Best parameters: {'contamination': 0.1, 'n_clusters': 3, 'n_estimators': 50, 'variance_ratio': 0.9}


In [123]:
tuner = kmeans_pipe_hyper_parameter_tuner(50, 0.1,0.9,3 , df=no_missing, random_state=42)
tuner.fit()
best_k_means_model = tuner.model

now lets tune dbscan hyper parameters

In [133]:
class dbscan_pipe_hyper_parameter_tuner:
    
    def __init__(self, n_estimators, contamination, variance_ratio, eps, mean_samples, df, random_state):
        self.n_estimators = n_estimators
        self.contamination = contamination
        self.variance_ratio = variance_ratio
        self.eps = eps
        self.mean_samples = mean_samples
        self.df = df
        self.random_state=random_state
        
    def fit(self):
        df_no_outliers, df_outliers = detect_outliers(self.df, self.n_estimators, self.contamination, self.random_state)
        df_normalized = normalize_features(df_no_outliers)
        df_pca = apply_pca(df_normalized, self.variance_ratio, self.random_state)
        dbscan_df , dbscan_model = fit_dbscan(df_pca , self.eps, self.mean_samples)
        self.model = dbscan_model
        self.PCA_df = df_pca

In [ ]:
param_grid = {
    'n_estimators': [25, 50],
    'contamination': [0.05, 0.1],
    'variance_ratio': [0.9, 0.95],
    'eps': [0.2, 0.3, 0.4],
    'mean_samples' : [10, 15, 20]
}

grid = ParameterGrid(param_grid)

best_score = -1
best_params = None

pbar = tqdm(total=len(grid), desc='Hyperparameter Tuning', ncols=80)

for params in grid:
    tuner = dbscan_pipe_hyper_parameter_tuner(**params, df=no_missing, random_state=42)
    tuner.fit()
    score = silhouette_score(tuner.PCA_df, tuner.model.labels_)
    
    print (f'for params {params} got the score of : {score}')
    
    if score > best_score:
        best_score = score
        best_params = params

    pbar.update()

pbar.close()

print(f"Best score: {best_score}")
print(f"Best parameters: {best_params}")

In [172]:
from tqdm import tqdm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.metrics import silhouette_samples
import numpy as np

class kmeans_pipe_hyper_parameter_tuner:
    
    def __init__(self, n_estimators, contamination, variance_ratio, n_clusters, df, random_state):
        self.n_estimators = n_estimators
        self.contamination = contamination
        self.variance_ratio = variance_ratio
        self.n_clusters = n_clusters
        self.df = df
        self.random_state=random_state
        
    def fit(self):
        df_no_outliers, df_outliers = detect_outliers(self.df, self.n_estimators, self.contamination, self.random_state)
        df_normalized = normalize_features(df_no_outliers)
        df_pca = apply_pca(df_normalized, variance_ratio = self.variance_ratio, random_state = self.random_state)
        K_means_df , K_means_modle = fit_kmeans(df_pca , n_clusters = self.n_clusters, random_state = self.random_state)
        self.model = K_means_modle
        self.PCA_df = df_pca
        self.labels_ = self.model.labels_

    def sample_silhouette_score(self, sample_percentage):
        unique_labels = np.unique(self.labels_)
        sample_indices = []

        for label in unique_labels:
            cluster_indices = np.where(self.labels_ == label)[0]
            sample_size = int(len(cluster_indices) * sample_percentage)
            if cluster_indices.size > 0:
                sample_indices.extend(np.random.choice(cluster_indices, size=min(sample_size, cluster_indices.size), replace=False))

        sample_labels = self.labels_[sample_indices]
        sample_data = self.PCA_df.reset_index(drop=True).iloc[sample_indices]

        return silhouette_score(sample_data, sample_labels)

    def cv_silhouette_score(self, sample_percentage, cv):
        scores = []
        for _ in range(cv):
            score = self.sample_silhouette_score(sample_percentage)
            scores.append(score)
        return np.mean(scores)

from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score
from tqdm import tqdm

param_grid = {
    'n_estimators': [50,100],
    'contamination': [0.1, 0.15, 0.20],
    'variance_ratio': [3, 5],
    'n_clusters': [3, 4, 5, 6]
}

grid = ParameterGrid(param_grid)

best_score = -1
best_params = None

pbar = tqdm(total=len(grid), desc='Hyperparameter Tuning', ncols=80)

for params in grid:
    tuner = kmeans_pipe_hyper_parameter_tuner(**params, df=no_missing, random_state=1)
    tuner.fit()
    score = tuner.cv_silhouette_score(sample_percentage=0.1, cv=3)  # Adjust sample percentage and cv as needed

    print (f'for params {params} got the score of : {score}')

    if score > best_score:
        best_score = score
        best_params = params

    pbar.update()

pbar.close()

print(f"Best score: {best_score}")
print(f"Best parameters: {best_params}")


Hyperparameter Tuning:  38%|███████▌            | 18/48 [02:14<03:43,  7.46s/it]


in outlier detection n_estimators is 50 and the contamination is 0.1
in pca the variance ratio is 3
Total explained variance: 0.9999999999850945
in k-means the n_clusters is 3


for params {'contamination': 0.1, 'n_clusters': 3, 'n_estimators': 50, 'variance_ratio': 3} got the score of : 0.273004092007024
in outlier detection n_estimators is 50 and the contamination is 0.1
in pca the variance ratio is 5
Total explained variance: 0.9999999999999462
in k-means the n_clusters is 3


for params {'contamination': 0.1, 'n_clusters': 3, 'n_estimators': 50, 'variance_ratio': 5} got the score of : 0.16071018048520405
in outlier detection n_estimators is 100 and the contamination is 0.1
in pca the variance ratio is 3
Total explained variance: 0.9999999999852016
in k-means the n_clusters is 3


for params {'contamination': 0.1, 'n_clusters': 3, 'n_estimators': 100, 'variance_ratio': 3} got the score of : 0.2589098171806015
in outlier detection n_estimators is 100 and the contamination is 0.1
in pca the variance ratio is 5
Total explained variance: 0.9999999999999438
in k-means the n_clusters is 3


...

for params {'contamination': 0.1, 'n_clusters': 6, 'n_estimators': 100, 'variance_ratio': 5} got the score of : 0.1749376634502625
in outlier detection n_estimators is 50 and the contamination is 0.15
in pca the variance ratio is 3
Total explained variance: 0.9999999999853221
in k-means the n_clusters is 3


for params {'contamination': 0.15, 'n_clusters': 3, 'n_estimators': 50, 'variance_ratio': 3} got the score of : 0.2736586177983071
in outlier detection n_estimators is 50 and the contamination is 0.15
in pca the variance ratio is 5
Total explained variance: 0.9999999999999452
in k-means the n_clusters is 3


for params {'contamination': 0.15, 'n_clusters': 3, 'n_estimators': 50, 'variance_ratio': 5} got the score of : 0.14259322566343147
in outlier detection n_estimators is 100 and the contamination is 0.15
in pca the variance ratio is 3
Total explained variance: 0.9999999999854868
in k-means the n_clusters is 3


for params {'contamination': 0.15, 'n_clusters': 3, 'n_estimators': 100, 'variance_ratio': 3} got the score of : 0.29075773250693954
in outlier detection n_estimators is 100 and the contamination is 0.15


KeyboardInterrupt: 

In [156]:
gc.collect()
tuner = kmeans_pipe_hyper_parameter_tuner(contamination=0.2, n_clusters=4, n_estimators= 100, variance_ratio=3, df=no_missing, random_state=1)
tuner.fit()
score = silhouette_score(tuner.PCA_df, tuner.model.labels_)
print (score)

in outlier detection n_estimators is 100 and the contamination is 0.2
in pca the variance ratio is 3
Total explained variance: 0.21385297021064714
in k-means the n_clusters is 4
0.2839700722573416


In [132]:
print (tuner.PCA_df.shape)

(81410, 26)


In [169]:
from tqdm import tqdm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.metrics import silhouette_samples
import numpy as np

class dbscan_pipe_hyper_parameter_tuner:
    
    def __init__(self, n_estimators, contamination, variance_ratio, eps , min_samples, df, random_state):
        self.n_estimators = n_estimators
        self.contamination = contamination
        self.variance_ratio = variance_ratio
        self.eps = eps
        self.df = df
        self.random_state=random_state
        self.min_samples = min_samples
        
    def fit(self):
        df_no_outliers, df_outliers = detect_outliers(self.df, self.n_estimators, self.contamination, self.random_state)
        df_normalized = normalize_features(df_no_outliers)
        df_pca = apply_pca(df_normalized, variance_ratio = self.variance_ratio, random_state = self.random_state)
        K_means_df , K_means_modle = fit_dbscan(eps= self.eps, min_samples= self.min_samples, df = df_pca)
        self.model = K_means_modle
        self.PCA_df = df_pca
        self.labels_ = self.model.labels_

    def sample_silhouette_score(self, sample_percentage):
        unique_labels = np.unique(self.labels_)
        if len(unique_labels) < 2:
            return -1

        sample_indices = []
        for label in unique_labels:
            cluster_indices = np.where(self.labels_ == label)[0]
            sample_size = int(len(cluster_indices) * sample_percentage)
            if cluster_indices.size > 0:
                sample_indices.extend(np.random.choice(cluster_indices, size=min(sample_size, cluster_indices.size), replace=False))

        sample_labels = self.labels_[sample_indices]
        sample_data = self.PCA_df.reset_index(drop=True).iloc[sample_indices]

        return silhouette_score(sample_data, sample_labels)

    def cv_silhouette_score(self, sample_percentage, cv):
        scores = []
        for _ in range(cv):
            score = self.sample_silhouette_score(sample_percentage)
            scores.append(score)
        return np.mean(scores)

from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score
from tqdm import tqdm

param_grid = {
    'n_estimators': [100],
    'contamination': [0.20],
    'variance_ratio': [3, 5],
    'eps': [0.6, 0.7, 0.8, 0.9],
    'min_samples': [5, 10, 15, 20, 25]
}

grid = ParameterGrid(param_grid)

best_score = -1
best_params = None

pbar = tqdm(total=len(grid), desc='Hyperparameter Tuning', ncols=80)

for params in grid:
    tuner = dbscan_pipe_hyper_parameter_tuner(**params, df=no_missing, random_state=1)
    tuner.fit()
    score = tuner.cv_silhouette_score(sample_percentage=0.1, cv=3)  # Adjust sample percentage and cv as needed

    print (f'for params {params} got the score of : {score}')

    if score > best_score:
        best_score = score
        best_params = params

    pbar.update()

pbar.close()

print(f"Best score: {best_score}")
print(f"Best parameters: {best_params}")


Hyperparameter Tuning:   0%|                             | 0/40 [00:00<?, ?it/s]

in outlier detection n_estimators is 100 and the contamination is 0.2
in pca the variance ratio is 3
Total explained variance: 0.21385297021064714


Hyperparameter Tuning:   2%|▌                    | 1/40 [00:14<09:29, 14.60s/it]

for params {'contamination': 0.2, 'eps': 0.6, 'min_samples': 5, 'n_estimators': 100, 'variance_ratio': 3} got the score of : 0.502369925505771
in outlier detection n_estimators is 100 and the contamination is 0.2
in pca the variance ratio is 5
Total explained variance: 0.305490242784654


Hyperparameter Tuning:   5%|█                    | 2/40 [00:26<08:22, 13.23s/it]

...

for params {'contamination': 0.2, 'eps': 0.9, 'min_samples': 20, 'n_estimators': 100, 'variance_ratio': 5} got the score of : 0.2502740654173078
in outlier detection n_estimators is 100 and the contamination is 0.2
in pca the variance ratio is 3
Total explained variance: 0.21385297021064714


Hyperparameter Tuning:  98%|███████████████████▌| 39/40 [10:11<00:18, 18.66s/it]

for params {'contamination': 0.2, 'eps': 0.9, 'min_samples': 25, 'n_estimators': 100, 'variance_ratio': 3} got the score of : 0.4739176663764398
in outlier detection n_estimators is 100 and the contamination is 0.2
in pca the variance ratio is 5
Total explained variance: 0.305490242784654


Hyperparameter Tuning: 100%|████████████████████| 40/40 [10:28<00:00, 15.72s/it]

for params {'contamination': 0.2, 'eps': 0.9, 'min_samples': 25, 'n_estimators': 100, 'variance_ratio': 5} got the score of : 0.27872941144193697
Best score: 0.502369925505771
Best parameters: {'contamination': 0.2, 'eps': 0.6, 'min_samples': 5, 'n_estimators': 100, 'variance_ratio': 3}


In [168]:
gc.collect()
tuner = dbscan_pipe_hyper_parameter_tuner(contamination=0.2, min_samples=10, n_estimators= 100, variance_ratio=3, eps=0.6, df=no_missing, random_state=1)
tuner.fit()
score = silhouette_score(tuner.PCA_df, tuner.model.labels_)
print (score)

in outlier detection n_estimators is 100 and the contamination is 0.2
in pca the variance ratio is 3
Total explained variance: 0.21385297021064714
0.41858500490492595


In [170]:
gc.collect()
tuner = dbscan_pipe_hyper_parameter_tuner(contamination=0.2, min_samples=5, n_estimators= 100, variance_ratio=3, eps=0.6, df=no_missing, random_state=1)
tuner.fit()
score = silhouette_score(tuner.PCA_df, tuner.model.labels_)
print (score)

in outlier detection n_estimators is 100 and the contamination is 0.2
in pca the variance ratio is 3
Total explained variance: 0.21385297021064714
0.4346444778621236
